# Data Import

## Introduction

This notebook shows how to import data into a DuckDB database. It uses the `duckdb` Python package to connect to a DuckDB database and import data from various formats, including CSV, JSON, DataFrame, parquet, GeoJSON, Shapefile, GeoParquet, and more.

## Datasets

The following datasets are used in this notebook. You don't need to download them, they can be accessed directly from the notebook.

- [cities.csv](https://open.gishub.org/data/duckdb/cities.csv)
- [countries.csv](https://open.gishub.org/data/duckdb/countries.csv)

## Installation

Uncomment the following cell to install the required packages if needed.

In [ ]:
# %pip install duckdb leafmap

## Library Import

In [ ]:
import duckdb
import leafmap
import pandas as pd

## Installing Extensions

DuckDB’s Python API provides functions for installing and loading extensions, which perform the equivalent operations to running the `INSTALL` and `LOAD` SQL commands, respectively. An example that installs and loads the [httpfs extension](https://duckdb.org/docs/extensions/httpfs) looks like follows:

In [ ]:
con = duckdb.connect()

In [ ]:
con.install_extension("httpfs")
con.load_extension("httpfs")

In [ ]:
con.install_extension("spatial")
con.load_extension("spatial")

## Downloading Sample Data

In [ ]:
url = "https://open.gishub.org/data/duckdb/cities.zip"
leafmap.download_file(url, unzip=True)

## CSV Files

CSV files can be read using the `read_csv` function, called either from within Python or directly from within SQL. By default, the `read_csv` function attempts to auto-detect the CSV settings by sampling from the provided file.

In [ ]:
# read from a file using fully auto-detected settings
con.read_csv('cities.csv')

In [ ]:
# specify options on how the CSV is formatted internally
con.read_csv('cities.csv', header=True, sep=',')

In [ ]:
# use the (experimental) parallel CSV reader
con.read_csv('cities.csv', parallel=True)

In [ ]:
# directly read a CSV file from within SQL
con.sql("SELECT * FROM 'cities.csv'")

In [ ]:
# call read_csv from within SQL
con.sql("SELECT * FROM read_csv_auto('cities.csv')")

## JSON Files

JSON files can be read using the `read_json` function, called either from within Python or directly from within SQL. By default, the `read_json` function will automatically detect if a file contains newline-delimited JSON or regular JSON, and will detect the schema of the objects stored within the JSON file.

In [ ]:
# read from a single JSON file
con.read_json('cities.json')

In [ ]:
# directly read a JSON file from within SQL
con.sql("SELECT * FROM 'cities.json'")

In [ ]:
# call read_json from within SQL
con.sql("SELECT * FROM read_json_auto('cities.json')")

## DataFrames

DuckDB is automatically able to query a Pandas DataFrame.

In [ ]:
df = pd.read_csv('cities.csv')
df

In [ ]:
con.sql('SELECT * FROM df').fetchall()

## Parquet Files

Parquet files can be read using the `read_parquet` function, called either from within Python or directly from within SQL.

In [ ]:
# read from a single Parquet file
con.read_parquet('cities.parquet')

In [ ]:
# directly read a Parquet file from within SQL
con.sql("SELECT * FROM 'cities.parquet'")

In [ ]:
# call read_parquet from within SQL
con.sql("SELECT * FROM read_parquet('cities.parquet')")

## GeoJSON Files

In [ ]:
con.sql('SELECT * FROM ST_Drivers()')

In [ ]:
con.sql("SELECT * FROM ST_Read('cities.geojson')")

In [ ]:
con.sql("FROM ST_Read('cities.geojson')")

In [ ]:
con.sql("CREATE TABLE cities AS SELECT * FROM ST_Read('cities.geojson')")

In [ ]:
con.table('cities')

In [ ]:
con.sql("SELECT * FROM cities")

## Shapefiles

In [ ]:
con.sql("SELECT * FROM ST_Read('cities.shp')")

In [ ]:
con.sql("FROM ST_Read('cities.shp')")

In [ ]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS cities2 AS 
        SELECT * FROM ST_Read('cities.shp')
        """
)

In [ ]:
con.table('cities2')

In [ ]:
con.sql('SELECT * FROM cities2')

## GeoParquet Files

In [ ]:
con.sql("SELECT * FROM 'cities.parquet'")

In [ ]:
con.sql(
    """
CREATE TABLE IF NOT EXISTS cities3 AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) 
AS geometry FROM 'cities.parquet'
"""
)

In [ ]:
con.table('cities3')

In [ ]:
con.sql(
    """
CREATE TABLE IF NOT EXISTS country AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) FROM
        's3://us-west-2.opendata.source.coop/google-research-open-buildings/v2/geoparquet-admin1/country=SSD/*.parquet'
"""
)

In [ ]:
con.table('country')

In [ ]:
con.sql('SELECT COUNT(*) FROM country')

## References

- [DuckDB Data Ingestion](https://duckdb.org/docs/api/python/data_ingestion)